COMP 215 - LAB 2 Records (NEO)
----------------
#### Name:
#### Date:

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [9]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [10]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the stringg
print(formatted_date)

2024-01-16 <class 'datetime.date'>
Today is: 2024-01-16


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [19]:
API_KEY = 'lkh9yVreJRQafd5QYNFWaZvoMr2wsaVCIiXgLWhm'  # substitute your API key here
# str(datetime.date.today())
today = '2024-01-05' #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
# pprint(data)
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:5])

12 Near Earth Objects found for 2024-01-05
[{'absolute_magnitude_h': 18.53,
  'close_approach_data': [{'close_approach_date': '2024-01-05',
                           'close_approach_date_full': '2024-Jan-05 01:38',
                           'epoch_date_close_approach': 1704418680000,
                           'miss_distance': {'astronomical': '0.4075397025',
                                             'kilometers': '60967071.434433675',
                                             'lunar': '158.5329442725',
                                             'miles': '37883181.522887715'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '33517.450331986',
                                                 'kilometers_per_second': '9.31040287',
                                                 'miles_per_hour': '20826.4401798943'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 3837.2936060376,
       

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [18]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

2 potentially hazardous asteroids identified.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [21]:
# Ex. 1 your code here

hazards = [item for item in neos if item['is_potentially_hazardous_asteroid']]

pprint(hazards)

print(f'{len(hazards)} potentially hazardous asteroids identified.')

[{'absolute_magnitude_h': 18.53,
  'close_approach_data': [{'close_approach_date': '2024-01-05',
                           'close_approach_date_full': '2024-Jan-05 01:38',
                           'epoch_date_close_approach': 1704418680000,
                           'miss_distance': {'astronomical': '0.4075397025',
                                             'kilometers': '60967071.434433675',
                                             'lunar': '158.5329442725',
                                             'miles': '37883181.522887715'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '33517.450331986',
                                                 'kilometers_per_second': '9.31040287',
                                                 'miles_per_hour': '20826.4401798943'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 3837.2936060376,
                                  'estimated_diame

## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [22]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

{'absolute_magnitude_h': 18.53,
 'close_approach_data': [{'close_approach_date': '1900-02-21',
                          'close_approach_date_full': '1900-Feb-21 11:18',
                          'epoch_date_close_approach': -2204541720000,
                          'miss_distance': {'astronomical': '0.2566829433',
                                            'kilometers': '38399221.583010771',
                                            'lunar': '99.8496649437',
                                            'miles': '23860169.8809037998'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '68021.5009854118',
                                                'kilometers_per_second': '18.8948613848',
                                                'miles_per_hour': '42265.915431741'}},
                         {'close_approach_date': '1901-03-08',
                          'close_approach_date_full': '1901-Mar-08 11:15'

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [37]:
# Ex. 2 your code here
upper_list = data['close_approach_data']
# pprint(upper_list)
n_dates = [i['close_approach_date'] for i in upper_list if i['orbiting_body'] == 'Earth']

o_bodies = [i['orbiting_body'] for i in upper_list if i['orbiting_body'] == 'Earth']

final = tuple(zip(n_dates, o_bodies))
print(final)
# len(n_dates)==len(o_bodies)

(('1900-02-21', 'Earth'), ('1901-03-08', 'Earth'), ('1902-01-24', 'Earth'), ('1902-05-16', 'Earth'), ('1902-12-22', 'Earth'), ('1903-07-05', 'Earth'), ('1903-11-20', 'Earth'), ('1904-09-17', 'Earth'), ('1905-10-07', 'Earth'), ('1906-10-22', 'Earth'), ('1919-02-15', 'Earth'), ('1920-03-01', 'Earth'), ('1921-02-18', 'Earth'), ('1921-04-14', 'Earth'), ('1922-01-06', 'Earth'), ('1922-06-10', 'Earth'), ('1922-12-07', 'Earth'), ('1923-07-30', 'Earth'), ('1923-11-02', 'Earth'), ('1924-09-28', 'Earth'), ('1925-10-13', 'Earth'), ('1939-02-15', 'Earth'), ('1940-02-29', 'Earth'), ('1941-02-26', 'Earth'), ('1941-04-05', 'Earth'), ('1942-01-10', 'Earth'), ('1942-06-04', 'Earth'), ('1942-12-11', 'Earth'), ('1943-07-22', 'Earth'), ('1943-11-09', 'Earth'), ('1944-09-25', 'Earth'), ('1945-10-10', 'Earth'), ('1960-02-24', 'Earth'), ('1961-03-09', 'Earth'), ('1962-01-23', 'Earth'), ('1962-05-16', 'Earth'), ('1962-12-23', 'Earth'), ('1963-07-02', 'Earth'), ('1963-11-23', 'Earth'), ('1964-08-31', 'Earth'),

## Challenge - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [ ]:
# Ex. 3 (challenge) your code here